In [2]:
import requests
import json
import bs4
import pandas as pd
import numpy as np
import threading
from tqdm import tqdm

header ={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"}

cap=["nano,micro,small,medium,large,mega"]
exchange=["nasdaq,nyse,amex"]

In [3]:
allStockAddr="https://api.nasdaq.com/api/screener/stocks?tableonly=true&limit=1600&exchange=NASDAQ&marketcap=nano"

allStockData=requests.get(allStockAddr,headers=header,timeout=10).text

#print(allStockData)

allStockJson=json.loads(allStockData)
#print(allStockJson)
allStockJson=allStockJson["data"]["table"]["rows"]
stockCodes=[]
for firm in allStockJson:
    stockCodes.append(firm["symbol"])
print(stockCodes)


['BRFH', 'SPAI', 'XTKG', 'BNR', 'CETY', 'CLST', 'CPTN', 'CRWS', 'BLRX', 'RRGB', 'TTOO', 'ANEB', 'CPBI', 'HOUR', 'FLUX', 'AIRTP', 'SWVL', 'RPHM', 'NISN', 'OABIW', 'LEE', 'CMBM', 'FKWL', 'MESA', 'ARQQ', 'RECT', 'FLNT', 'DRRX', 'PPSI', 'RLYB', 'AWRE', 'RGS', 'NTRB', 'ESGL', 'BWEN', 'ATRA', 'OSS', 'TCBS', 'EOSEW', 'OPTX', 'BGFV', 'CREX', 'PFXNZ', 'VTVT', 'SYT', 'DWSN', 'BGM', 'APYX', 'UCL', 'EHGO', 'AADI', 'DXR', 'LVTX', 'NAAS', 'ELTX', 'NICK', 'LVROW', 'LPTH', 'OPHC', 'FSEA', 'CCG', 'HSON', 'ICMB', 'EYEN', 'CMCT', 'CFSB', 'PASG', 'LUCD', 'ARKR', 'ASRV', 'TACT', 'CTRM', 'PLBY', 'RAND', 'RVPH', 'CODX', 'SELX', 'FEAM', 'OCX', 'KPLT', 'PHUN', 'KZR', 'PDYN', 'CARM', 'LINK', 'FBIO', 'USIO', 'SYRS', 'MNY', 'RCON', 'FATBW', 'TRSG', 'NAII', 'ARKOW', 'TARA', 'ESLA', 'PBBK', 'PET', 'HUIZ', 'IDN', 'FGEN', 'HFBL', 'PODC', 'SJ', 'ECOR', 'DRCT', 'RENT', 'SWIN', 'CCLD', 'XOS', 'RDI', 'INTG', 'DGHI', 'AKTX', 'BNGO', 'CLIR', 'RFIL', 'ECDA', 'NXTC', 'DYAI', 'EBON', 'AXDX', 'MNDO', 'YYGH', 'DSWL', 'ICAD', 'S

In [12]:
yahoofinanceAPIBaseLink="https://query2.finance.yahoo.com/v8/finance/chart/"
stock=""
start_time="1725652200"
end_time="1725904800"
interval="1m"

threads_num=64
threads=[]
fullStockInfo=[]
# stockCodes=np.array(stockCodes)
# stockSplit=np.split(stockCodes,4,)
splitStockCode=[]
divNumb=int(len(stockCodes)/threads_num)
for i in range(0,threads_num):
    if(i!=(threads_num-1)):
        splitStockCode.append(stockCodes[divNumb*i:divNumb*(i+1)])
    else:
        splitStockCode.append(stockCodes[divNumb*i:len(stockCodes)])

def getStockData(code):
    yahoofinanceAPIUrl=yahoofinanceAPIBaseLink+code+"?"+"period1="+start_time+"&period2="+end_time+"&interval="+interval+"&includePrePost=true&lang=en-US&region=US"
    #print(yahoofinanceAPIUrl)
    stockData=requests.get(yahoofinanceAPIUrl,headers=header,timeout=5)
    fullStockInfo.append(stockData.text)
    #print(stockData)

def worker(codes,thread_numb):
    progress_bar = tqdm(codes,desc=f"Thread {thread_numb} Progress", position=thread_numb, leave=True,dynamic_ncols=False)
    for code in codes:
        getStockData(code)
        progress_bar.update(1)

for i in range(threads_num):
    thread=threading.Thread(target=worker,args=(splitStockCode[i],i))
    threads.append(thread)

for thread in threads:
    thread.start()

for thread in threads:
    thread.join()


Thread 0 Progress:   0%|          | 0/21 [00:00<?, ?it/s]




























































































Thread 0 Progress:   5%|▍         | 1/21 [00:00<00:03,  5.60it/s]






























































































































































Thread 0 Progress:  10%|▉         | 2/21 [00:00<00:06,  2.89it/s]










































































































































































Thread 0 Progress:  14%|█▍        | 3/21 [00:01<00:07,  2.31it/s]






















































































































































































Thread 0 Progress:  19%|█▉        | 4/21 [00:02<00:08,  1.93it/s]







































































































































